# Download the data and convert it to netCDF

The original data is provided in the supplementary material of the paper as a text file of longitude, latitude, Moho depth, and uncertainty. It's much easier to work with the gridded data if it's stored in netCDF instead.

## Import things

In [1]:
import numpy as np
import xarray as xr
import pooch

## Download

We'll download the data using [Pooch](https://github.com/fatiando/pooch). It will be saved to a cache folder on your computer that it's only downloaded once, even if you run this code many times.

In [2]:
url = "https://agupubs.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1029%2F2018JB016593&file=jgrb53251-sup-0003-Data_Set_SI-S03.txt"
# Pretent to be a browser so AGU lets us download the file
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}
fname = pooch.retrieve(
    url, 
    known_hash="sha256:1a8654f3b8f0821b30a1f1fc63034450c37b282e9f2016f14b075853e305b13e", 
    fname="szwillus-moho-depth.txt",
    downloader=pooch.HTTPDownloader(headers=headers)
)

## Read

Read in the data using numpy.